In [1]:
 # Dependencies
# import numpy as np
import pandas as pd
import requests
from census import Census

# Census API Key
from config import api_key
c = Census(api_key, year=2016)

In [2]:
#make API call to Census API for all US county estimated population for 2016
county_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "county:*"})

In [4]:
#create a df with the county pop data
county_pop_df = pd.DataFrame(county_pop_data)
county_pop_df.head()

,NAME,B01001_001E,state,county
0,"Carroll County, Arkansas",27690,05,015
1,"Chicot County, Arkansas",11189,05,017
2,"Clark County, Arkansas",22684,05,019
3,"Clay County, Arkansas",15202,05,021
4,"Cleburne County, Arkansas",25587,05,023


In [10]:
#make another call for state population just to get a list of states since county infor only have state census id
state_pop_data= c.acs5.get(("NAME","B01001_001E"), {"for": "state:*"})

In [11]:
#pass on the state pop data to a df
state_pop_df = pd.DataFrame(state_pop_data)
state_pop_df.head()

,NAME,B01001_001E,state
0,Alabama,4841164,01
1,Alaska,736855,02
2,Arizona,6728577,04
3,Arkansas,2968472,05
4,California,38654206,06


In [13]:
#drop population column from the state pop data df
state_pop1_df = state_pop_df.drop(columns=["B01001_001E"])

In [14]:
state_pop1_df.head()

,NAME,state
0,Alabama,01
1,Alaska,02
2,Arizona,04
3,Arkansas,05
4,California,06


In [15]:
#rename "Name" column to prevent duplication when merging with county pop df
state_pop1_df.rename(columns = {"NAME":"State_Name"}, inplace=True)

In [16]:
state_pop1_df.head()

,State_Name,state
0,Alabama,01
1,Alaska,02
2,Arizona,04
3,Arkansas,05
4,California,06


In [17]:
#merge state and county df on the "state" the numeric identifier for states
county_pop1_df = pd.merge(county_pop_df, state_pop1_df, on="state")

In [18]:
county_pop1_df.head()

,NAME,B01001_001E,state,county,State_Name
0,"Carroll County, Arkansas",27690,05,015,Arkansas
1,"Chicot County, Arkansas",11189,05,017,Arkansas
2,"Clark County, Arkansas",22684,05,019,Arkansas
3,"Clay County, Arkansas",15202,05,021,Arkansas
4,"Cleburne County, Arkansas",25587,05,023,Arkansas


In [19]:
#rename column names 
county_pop1_df.rename(columns = {"NAME":"county", "B01001_001E":"population","state":"stateId","county":"countyId", 
                                "State_Name":"state"}, inplace=True)

In [20]:
#This will be the 2016 Estimated Pop by County Table
county_pop1_df.head()

,county,population,stateId,countyId,state
0,"Carroll County, Arkansas",27690,05,015,Arkansas
1,"Chicot County, Arkansas",11189,05,017,Arkansas
2,"Clark County, Arkansas",22684,05,019,Arkansas
3,"Clay County, Arkansas",15202,05,021,Arkansas
4,"Cleburne County, Arkansas",25587,05,023,Arkansas


In [21]:
#import US state 2 letter abbreviation dictionary to create a state table 
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}

In [22]:
state_series = pd.Series(us_state_abbrev)

In [23]:
state_df = state_series.to_frame()

In [24]:
state_df.head()

,0
Alabama,AL
Alaska,AK
American Samoa,AS
Arizona,AZ
Arkansas,AR


In [25]:
state_df.reset_index(inplace=True)

In [26]:
state_df.head()

,index,0
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [27]:
state_df.rename(columns={"index":"state",0:"statecode"}, inplace=True)

In [28]:
#Create a state table
state_df.head()

,state,statecode
0,Alabama,AL
1,Alaska,AK
2,American Samoa,AS
3,Arizona,AZ
4,Arkansas,AR


In [31]:
state_pop_df.drop(columns=["B01001_001E"])
state_pop_df.rename(columns = {"NAME":"state", "state":"stateId"}, inplace=True)


In [32]:
state1_df = pd.merge(state_df, state_pop_df, on="state")

In [33]:
state1_df.head()

,state,statecode,B01001_001E,stateId
0,Alabama,AL,4841164,01
1,Alaska,AK,736855,02
2,Arizona,AZ,6728577,04
3,Arkansas,AR,2968472,05
4,California,CA,38654206,06


In [37]:

state1_df.drop(columns=["B01001_001E"], inplace=True)

In [38]:
#STATE TABLE
state1_df.head()

,state,statecode,stateId
0,Alabama,AL,01
1,Alaska,AK,02
2,Arizona,AZ,04
3,Arkansas,AR,05
4,California,CA,06
